# MiBici dataset
### 2014-2025 rides
This project analyzes ten years of public bike usage in Guadalajara using the MiBici dataset (2014–2024). By combining trip data with station information, the goal is to understand:
- how ridership patterns have evolved before and after the pandemic, 
- how usage differs by gender and age, 
- and how stations connect neighborhoods with different socioeconomic profiles. 

The final deliverables include a reproducible Python workflow and a public dashboard showcasing key insights.

## Project Guide

- Setup
    - Clean repo, virtual environment, basic libraries, sample dataset.

- Data Cleaning
    - Standardize column names, parse dates, create time features, clean station names.

- Data Enrichment
    - Merge trips with station metadata; add geographic or zone information if available.

- Exploratory Analysis
    - Trips by day and month, 
    - pre/post pandemic trends, 
    - gender patterns by station, 
    - neighborhood connectivity, 
    - usage by age and hour.

- Visualizations
    - Timeline charts, distribution plots, station-level patterns.

- Dashboard
    - Export processed data and build a clear, visual Tableau Public dashboard.

- Final Artifacts
    - Clean notebook, 
    - documented code, 
    - dashboard link, and 
    - concise project summary.

In [1]:
# Importing libraries 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
from datetime import datetime

In [2]:
# Importing databases
# 3,000 rows per year * 10
df_base = pd.read_csv('/Users/pablo/Desktop/temp/DA/miBici/mibici_2025/data/raw/mibici_2014-2024.csv', encoding="latin1", nrows = 30000)

In [35]:
import os
os.getcwd()


'/Users/pablo/Desktop/temp/DA/miBici/mibici_2025/notebooks'

In [3]:
df_base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Unnamed: 0      30000 non-null  int64 
 1   Trip_Id         30000 non-null  int64 
 2   User_Id         30000 non-null  int64 
 3   Sex             30000 non-null  object
 4   Birth_year      30000 non-null  int64 
 5   Trip_start      30000 non-null  object
 6   Trip_end        30000 non-null  object
 7   Origin_Id       30000 non-null  int64 
 8   Destination_Id  30000 non-null  int64 
 9   Age             30000 non-null  int64 
 10  Duration        30000 non-null  object
dtypes: int64(7), object(4)
memory usage: 2.5+ MB


In [4]:
#duplicates
print(df_base.duplicated().sum())
print(df_base['Trip_Id'].duplicated().sum())

0
0


In [5]:
# Merging 2025 data
paths_2025 = glob.glob("/Users/pablo/Desktop/temp/DA/miBici/mibici_2025/data/raw/datos_abiertos_2025_*.csv")
paths_2025

['/Users/pablo/Desktop/temp/DA/miBici/mibici_2025/data/raw/datos_abiertos_2025_08.csv',
 '/Users/pablo/Desktop/temp/DA/miBici/mibici_2025/data/raw/datos_abiertos_2025_09.csv',
 '/Users/pablo/Desktop/temp/DA/miBici/mibici_2025/data/raw/datos_abiertos_2025_01.csv',
 '/Users/pablo/Desktop/temp/DA/miBici/mibici_2025/data/raw/datos_abiertos_2025_02.csv',
 '/Users/pablo/Desktop/temp/DA/miBici/mibici_2025/data/raw/datos_abiertos_2025_03.csv',
 '/Users/pablo/Desktop/temp/DA/miBici/mibici_2025/data/raw/datos_abiertos_2025_07.csv',
 '/Users/pablo/Desktop/temp/DA/miBici/mibici_2025/data/raw/datos_abiertos_2025_06.csv',
 '/Users/pablo/Desktop/temp/DA/miBici/mibici_2025/data/raw/datos_abiertos_2025_04.csv',
 '/Users/pablo/Desktop/temp/DA/miBici/mibici_2025/data/raw/datos_abiertos_2025_10.csv',
 '/Users/pablo/Desktop/temp/DA/miBici/mibici_2025/data/raw/datos_abiertos_2025_05.csv']

In [6]:
# Encoding dataframes, normalizing column names
# 250 rows per month = 3000 per year

dfs = []
for p in paths_2025:
    df = pd.read_csv(p, nrows= 250, encoding="latin1")
    # Normalizamos nombres de columnas
    if "AÃ±o_de_nacimiento" in df.columns:
        df = df.rename(columns={"AÃ±o_de_nacimiento": "Año_de_nacimiento"})
    dfs.append(df)

# concatenamos todos los archivos ya normalizados
df_2025 = pd.concat(dfs, ignore_index=True)


In [7]:
# Rename columns
df_2025 = df_2025.rename(columns= {'Viaje_Id': 'Trip_Id', 'Usuario_Id': 'User_Id', 'Genero': 'Sex', 'Año_de_nacimiento': 'Birth_year', 'Inicio_del_viaje': 'Trip_start', 'Fin_del_viaje': 'Trip_end', 'Origen_Id': 'Origin_Id', 'Destino_Id': 'Destination_Id'})

In [8]:
df_2025.head()

,Trip_Id,User_Id,Sex,Birth_year,Trip_start,Trip_end,Origin_Id,Destination_Id
0,40200545,461522,M,2001,2025-08-01 00:00:05,2025-08-01 00:14:13,263,58
1,40200547,434514,M,1994,2025-08-01 00:00:30,2025-08-01 00:21:02,161,76
2,40200549,474257,F,1996,2025-08-01 00:00:45,2025-08-01 00:15:35,389,286
3,40200550,601717,M,1986,2025-08-01 00:01:02,2025-08-01 00:05:40,36,384
4,40200551,2752555,F,2003,2025-08-01 00:01:08,2025-08-01 00:08:09,150,152


In [9]:
df_2025.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Trip_Id         2500 non-null   int64 
 1   User_Id         2500 non-null   int64 
 2   Sex             2473 non-null   object
 3   Birth_year      2500 non-null   int64 
 4   Trip_start      2500 non-null   object
 5   Trip_end        2500 non-null   object
 6   Origin_Id       2500 non-null   int64 
 7   Destination_Id  2500 non-null   int64 
dtypes: int64(5), object(3)
memory usage: 156.4+ KB


In [10]:
# Duplicates
print(df_2025.duplicated().sum())
print(df_2025['Trip_Id'].duplicated().sum())

0
0


In [11]:
# Nans search
nans_in_birthYear = df_2025['Birth_year'].isna().sum()
nans_sex = df_2025['Sex'].isna().sum()
print(f"There's {nans_in_birthYear} NANs in Birth_year")
print(f"There's {nans_sex} NANs in Sex")

There's 0 NANs in Birth_year
There's 27 NANs in Sex


In [12]:
# NAs of Sex
df_2025['Sex'] = df_2025['Sex'].fillna('NA')

In [13]:
# Getting column Age
df_2025['Birth_year'] = df_2025['Birth_year'].astype('Int64')
df_2025['Age'] = (datetime.now().year - df_2025['Birth_year']).astype('Int64')

In [14]:
df_2025.head(3)

,Trip_Id,User_Id,Sex,Birth_year,Trip_start,Trip_end,Origin_Id,Destination_Id,Age
0,40200545,461522,M,2001,2025-08-01 00:00:05,2025-08-01 00:14:13,263,58,24
1,40200547,434514,M,1994,2025-08-01 00:00:30,2025-08-01 00:21:02,161,76,31
2,40200549,474257,F,1996,2025-08-01 00:00:45,2025-08-01 00:15:35,389,286,29


In [15]:
df_full = pd.concat([df_base, df_2025], ignore_index=True)
df_full.drop_duplicates(subset='Trip_Id')

,Unnamed: 0,Trip_Id,User_Id,Sex,Birth_year,Trip_start,Trip_end,Origin_Id,Destination_Id,Age,Duration
0,0.0,32244893,1470734,M,1981,2024-01-31 23:59:33,2024-02-01 00:11:15,24,86,43,0 days 00:11:42
1,1.0,32244892,2731702,M,1994,2024-01-31 23:59:06,2024-02-01 00:10:49,48,279,30,0 days 00:11:43
2,2.0,32244891,1431452,M,2001,2024-01-31 23:58:48,2024-02-01 00:01:42,273,383,23,0 days 00:02:54
3,3.0,32244890,2312602,F,2003,2024-01-31 23:58:44,2024-02-01 00:01:58,273,383,21,0 days 00:03:14
4,4.0,32244889,2266427,M,1999,2024-01-31 23:58:44,2024-02-01 00:01:39,273,383,25,0 days 00:02:55
...,...,...,...,...,...,...,...,...,...,...,...
32495,NaN,38912149,715120,M,2001,2025-05-01 06:43:02,2025-05-01 06:47:36,307,265,24,NaN
32496,NaN,38912150,15604,M,1989,2025-05-01 06:43:12,2025-05-01 06:47:35,164,333,36,NaN
32497,NaN,38912151,4547334,F,2000,2025-05-01 06:43:14,2025-05-01 06:47:33,130,125,25,NaN
32498,NaN,38912152,4523643,NA,2002,2025-05-01 06:44:13,2025-05-01 07:01:32,166,45,23,NaN


In [16]:
df_full.head(3)

,Unnamed: 0,Trip_Id,User_Id,Sex,Birth_year,Trip_start,Trip_end,Origin_Id,Destination_Id,Age,Duration
0,0.0,32244893,1470734,M,1981,2024-01-31 23:59:33,2024-02-01 00:11:15,24,86,43,0 days 00:11:42
1,1.0,32244892,2731702,M,1994,2024-01-31 23:59:06,2024-02-01 00:10:49,48,279,30,0 days 00:11:43
2,2.0,32244891,1431452,M,2001,2024-01-31 23:58:48,2024-02-01 00:01:42,273,383,23,0 days 00:02:54


In [17]:
df_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32500 entries, 0 to 32499
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      30000 non-null  float64
 1   Trip_Id         32500 non-null  int64  
 2   User_Id         32500 non-null  int64  
 3   Sex             32500 non-null  object 
 4   Birth_year      32500 non-null  Int64  
 5   Trip_start      32500 non-null  object 
 6   Trip_end        32500 non-null  object 
 7   Origin_Id       32500 non-null  int64  
 8   Destination_Id  32500 non-null  int64  
 9   Age             32500 non-null  Int64  
 10  Duration        30000 non-null  object 
dtypes: Int64(2), float64(1), int64(4), object(4)
memory usage: 2.8+ MB


In [18]:
df_full['Trip_start'] = pd.to_datetime(df_full['Trip_start'], errors='coerce')
df_full["Trip_start"].min(), df_full["Trip_start"].max()

(Timestamp('2024-01-30 06:28:40'), Timestamp('2025-10-01 06:13:50'))

In [19]:
# Eliminamos columna unnamed
df_full.drop('Unnamed: 0', axis=1, inplace=True)

In [20]:
# Cambiamos columnas a bajas
df_full.columns = df_full.columns.str.lower().str.replace(' ', '_').str.strip()

# Convertimos trip_end y trip_start a datetime
df_full.trip_end = pd.to_datetime(df_full.trip_end, errors=('coerce'))
df_full.trip_start = pd.to_datetime(df_full.trip_start, errors=('coerce'))

# Sacamos fecha del viaje, mes y año
df_full['date'] = df_full.trip_start.dt.to_period('M')

# Agregamos columna de duración del viaje, en minutos
df_full['duration_m'] = ((df_full.trip_end - df_full.trip_start).dt.total_seconds() / 60).round().astype(int)

df_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32500 entries, 0 to 32499
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   trip_id         32500 non-null  int64         
 1   user_id         32500 non-null  int64         
 2   sex             32500 non-null  object        
 3   birth_year      32500 non-null  Int64         
 4   trip_start      32500 non-null  datetime64[ns]
 5   trip_end        32500 non-null  datetime64[ns]
 6   origin_id       32500 non-null  int64         
 7   destination_id  32500 non-null  int64         
 8   age             32500 non-null  Int64         
 9   duration        30000 non-null  object        
 10  date            32500 non-null  period[M]     
 11  duration_m      32500 non-null  int64         
dtypes: Int64(2), datetime64[ns](2), int64(5), object(2), period[M](1)
memory usage: 3.0+ MB


In [21]:
# Importamos datos de estaciones
estaciones = pd.read_csv('/Users/pablo/Desktop/temp/DA/miBici/mibici_2025/data/raw/nomenclatura_2025_10.csv', encoding="latin1")
estaciones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 383 entries, 0 to 382
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         383 non-null    int64  
 1   name       383 non-null    object 
 2   obcn       383 non-null    object 
 3   location   383 non-null    object 
 4   latitude   383 non-null    float64
 5   longitude  383 non-null    float64
 6   status     383 non-null    object 
dtypes: float64(2), int64(1), object(4)
memory usage: 21.1+ KB


In [22]:
estaciones.head()

,id,name,obcn,location,latitude,longitude,status
0,2,(GDL-001) C. Epigmenio Glez./ Av. 16 de Sept.,GDL-001,POLÍGONO CENTRAL,20.666378,-103.348820,IN_SERVICE
1,3,(GDL-002) C. Colonias / Av. Niños héroes,GDL-002,POLÍGONO CENTRAL,20.667228,-103.366000,IN_SERVICE
2,4,(GDL-003) C. Vidrio / Av. Chapultepec,GDL-003,POLÍGONO CENTRAL,20.667690,-103.368252,IN_SERVICE
3,5,(GDL-004) C. Ghilardi /C. Miraflores,GDL-004,POLÍGONO CENTRAL,20.691847,-103.362549,IN_SERVICE
4,6,(GDL-005) C. San Diego /Calzada Independencia,GDL-005,POLÍGONO CENTRAL,20.681158,-103.339363,IN_SERVICE


In [23]:
estaciones.id.duplicated().sum()

np.int64(0)

In [24]:
estaciones.location.value_counts()

location
POLÍGONO CENTRAL     287
TLQ-CORREDORATLAS     49
ZAPOPAN CENTRO        47
Name: count, dtype: int64

In [25]:
estaciones.status.value_counts()

status
IN_SERVICE        366
NOT_IN_SERVICE     17
Name: count, dtype: int64

In [26]:
# limpiamos columna name
estaciones = estaciones.rename(columns={'obcn': 'station_code'})
estaciones["name"] = estaciones["name"].str.replace(r"\([^)]*\)\s*", "", regex=True)
estaciones.head()

,id,name,station_code,location,latitude,longitude,status
0,2,C. Epigmenio Glez./ Av. 16 de Sept.,GDL-001,POLÍGONO CENTRAL,20.666378,-103.348820,IN_SERVICE
1,3,C. Colonias / Av. Niños héroes,GDL-002,POLÍGONO CENTRAL,20.667228,-103.366000,IN_SERVICE
2,4,C. Vidrio / Av. Chapultepec,GDL-003,POLÍGONO CENTRAL,20.667690,-103.368252,IN_SERVICE
3,5,C. Ghilardi /C. Miraflores,GDL-004,POLÍGONO CENTRAL,20.691847,-103.362549,IN_SERVICE
4,6,C. San Diego /Calzada Independencia,GDL-005,POLÍGONO CENTRAL,20.681158,-103.339363,IN_SERVICE


In [27]:
# Creamos dataset de estaciones origin
df_origin = estaciones[['id', 'station_code']]
df_origin = df_origin.rename(columns= {'id': 'origin_id', 'station_code': 'station_code_origin'})
df_origin.head(3)

,origin_id,station_code_origin
0,2,GDL-001
1,3,GDL-002
2,4,GDL-003


In [28]:
# Dataset destination stations

df_destination = estaciones[['id', 'station_code']]
df_destination = df_destination.rename(columns={'id': 'destination_id', 'station_code': 'station_code_destination'})
df_destination.head(3)

,destination_id,station_code_destination
0,2,GDL-001
1,3,GDL-002
2,4,GDL-003


In [29]:
df_destination.head()

,destination_id,station_code_destination
0,2,GDL-001
1,3,GDL-002
2,4,GDL-003
3,5,GDL-004
4,6,GDL-005


In [30]:
# Join all tables

df_merged = pd.merge(df_full, df_origin, on='origin_id')
df_merged = pd.merge(df_merged, df_destination, on = 'destination_id')

df_merged.head()

,trip_id,user_id,sex,birth_year,trip_start,trip_end,origin_id,destination_id,age,duration,date,duration_m,station_code_origin,station_code_destination
0,32244893,1470734,M,1981,2024-01-31 23:59:33,2024-02-01 00:11:15,24,86,43,0 days 00:11:42,2024-01,12,GDL-022,GDL-084
1,32244892,2731702,M,1994,2024-01-31 23:59:06,2024-02-01 00:10:49,48,279,30,0 days 00:11:43,2024-01,12,GDL-046,GDL-206
2,32244891,1431452,M,2001,2024-01-31 23:58:48,2024-02-01 00:01:42,273,383,23,0 days 00:02:54,2024-01,3,GDL-200,GDL-240
3,32244890,2312602,F,2003,2024-01-31 23:58:44,2024-02-01 00:01:58,273,383,21,0 days 00:03:14,2024-01,3,GDL-200,GDL-240
4,32244889,2266427,M,1999,2024-01-31 23:58:44,2024-02-01 00:01:39,273,383,25,0 days 00:02:55,2024-01,3,GDL-200,GDL-240


In [31]:
df_merged.shape[0]

32487

In [32]:
df = df_merged[['trip_id', 'user_id', 'sex', 'age', 'date', 'station_code_origin', 'station_code_destination', 'duration_m']]
df.head(3)

,trip_id,user_id,sex,age,date,station_code_origin,station_code_destination,duration_m
0,32244893,1470734,M,43,2024-01,GDL-022,GDL-084,12
1,32244892,2731702,M,30,2024-01,GDL-046,GDL-206,12
2,32244891,1431452,M,23,2024-01,GDL-200,GDL-240,3


In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32487 entries, 0 to 32486
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype    
---  ------                    --------------  -----    
 0   trip_id                   32487 non-null  int64    
 1   user_id                   32487 non-null  int64    
 2   sex                       32487 non-null  object   
 3   age                       32487 non-null  Int64    
 4   date                      32487 non-null  period[M]
 5   station_code_origin       32487 non-null  object   
 6   station_code_destination  32487 non-null  object   
 7   duration_m                32487 non-null  int64    
dtypes: Int64(1), int64(3), object(3), period[M](1)
memory usage: 2.0+ MB


In [34]:
df.to_csv('')

FileNotFoundError: [Errno 2] No such file or directory: ''